# Facebook data 📈

First of all we need to fix encoding of message.json file. To do this, we will encode message to **latin-1** and decode to **utf8**.

In [ ]:
from PyQt5 import QtGui
from PyQt5 import QtGui, QtWidgets

import json
import os
from pathlib import Path 

def open_file():
    app = QtWidgets.QApplication([dir])
    fname = QtWidgets.QFileDialog.getOpenFileName(None, "Select a file...", '.', filter="All files (*)")
    return fname[0]
    
fname = open_file()
filename = '{}_fixed.json'.format(os.path.splitext(fname)[0])


print('Working ...')

file = open(fname, 'r')  # or change filename to 'message.json'

jsonToFix = json.load(file)

for i in jsonToFix['messages']:
    try:
        i['sender_name'] = i['sender_name'].encode('latin-1').decode('utf8')
        i['content'] = i['content'].encode('latin-1').decode('utf8')
    except Exception:
        pass

jsonToFix['title'] = jsonToFix['title'].encode('latin-1').decode('utf8')

with open(filename, 'w', encoding='utf8') as fout:
    json.dump(jsonToFix, fout, indent=2, ensure_ascii=False)

print('File created at {}'.format(filename))





Now we can start making some cool stats 💻 top **n messages** from conversation 📖

In [ ]:
import re
from itertools import islice


file = open(filename, 'r', encoding='utf8')

msg_json = json.load(file)

# Count every word

messagesAll = {}

for i in msg_json['messages']:
    try:
        tmp = []
       # tmp = re.findall(r"[\w']+", i['content']) not sure which is better 😶
        tmp = i['content'].split()
        for word in tmp:
            if word in messagesAll:
                messagesAll[word] += 1
            else:
                messagesAll[word] = 1
    except Exception:
        pass

sortedAll = sorted(messagesAll.items(), key=lambda kv: kv[1], reverse=True)

def first_n_from_iterable(n, iterable):
    """Return first n items of the iterable as a list"""
    return list(islice(iterable, n))

def get_top_messages(dict_msgs, n=10):
    """Return first n messages of dict"""

    print('\nTop {} :\n'.format(n))
    for key in first_n_from_iterable(n, dict_msgs):
        print(key)
        
get_top_messages(sortedAll, 5)


Top messages per month / year 📆 and create dictionary with month and total messages.

In [ ]:
import datetime

messages_per_month_dict = {}
messages_per_month_dict_A = {}
messages_per_month_dict_B = {}




def get_top_per_month(n=10):
    count = 0
    mssgs_A = 0
    mssgs_B = 0
    tmp_dic = {}
    messages_per_month = 0
    for i in msg_json['messages']:
        try:
            if str(datetime.datetime.fromtimestamp(float(msg_json['messages'][count]['timestamp_ms']) / 1000.0).strftime('%Y-%B')) == str(datetime.datetime.fromtimestamp(float(msg_json['messages'][count+1]['timestamp_ms']) / 1000.0).strftime('%Y-%B')):
                messages_per_month += 1
                if i['sender_name'] == msg_json['title']:
                    mssgs_A += 1
                else:
                    mssgs_B += 1
                        
                
                tmp = []
                tmp = i['content'].split()
                for word in tmp:
                    if word in tmp_dic:
                        tmp_dic[word] += 1
                    else:
                        tmp_dic[word] = 1
                        

            else:
                print('\n{} | {} \n'.format(datetime.datetime.fromtimestamp(
                    float(i['timestamp_ms']) / 1000.0).strftime('%Y-%B'), messages_per_month))
            
                sorted_month = sorted(
                    tmp_dic.items(), key=lambda kv: kv[1], reverse=True)
                  
                messages_per_month_dict[datetime.datetime.fromtimestamp(float(i['timestamp_ms']) / 1000.0).strftime('%Y/%m')] = [messages_per_month, first_n_from_iterable(1, sorted_month)[0][0]]
                messages_per_month_dict_A[datetime.datetime.fromtimestamp(float(i['timestamp_ms']) / 1000.0).strftime('%Y/%m')] = mssgs_A
                messages_per_month_dict_B[datetime.datetime.fromtimestamp(float(i['timestamp_ms']) / 1000.0).strftime('%Y/%m')] = mssgs_B

                for key in first_n_from_iterable(n, sorted_month):
                    print('{}'.format(key))
                    
                messages_per_month = 0
                mssgs_A = 0
                mssgs_B = 0
                tmp_dic = {}
                sorted_month = {}
            count += 1
            
        except Exception:
            pass
        
get_top_per_month(5)

Top 5 per sender #TODO group chat 💥 

In [ ]:
messagesA = {}
messagesB = {}

for i in msg_json['messages']:
    try:
        tmp = []
       # tmp = re.findall(r"[\w']+", i['content']) # dont know which is better
        tmp = i['content'].split()
        for word in tmp:
            if i['sender_name'] == msg_json['title']:
                if word in messagesA:
                    messagesA[word] += 1
                else:
                    messagesA[word] = 1
            else:
                if word in messagesB:
                    messagesB[word] += 1
                else:
                    messagesB[word] = 1

    except Exception:
        pass
    
sortedA = sorted(messagesA.items(), key=lambda kv: kv[1], reverse=True)
sortedB = sorted(messagesB.items(), key=lambda kv: kv[1], reverse=True)
    
def get_top_messages_per_sender(dictA, dictB, n=10):
    """Return first n messages per sender"""

    print('\nTop {} from : {}'.format(n, msg_json['title']))
    for key in first_n_from_iterable(n, dictA):
        print(key)

    print('\nYour top {} '.format(n))
    for key in first_n_from_iterable(n, dictB):
        print(key)
        
get_top_messages_per_sender(sortedA, sortedB, 5)

If your conversation contains some youtube videos (of cource does 😉) you can search for them and their titles! ▶

In [ ]:
from urllib.request import urlopen
from lxml import etree

def get_youtube_titles_from_conversation():
 # delete to run this, otherwise its slow af
    videoCount = 0
    for i in msg_json['messages']:
        try:
            tmp = []
            tmp = i['content'].split()
            for word in tmp:
                if 'youtube.com/watch' in word:
                    try:
                        youtube = etree.HTML(urlopen(word).read())
                        video_title = youtube.xpath(
                            "//span[@id='eow-title']/@title")
                    except Exception as e:
                        print(e)
                        video_title = 'Error'

                    print('{} -> {}'.format(word, video_title))
                    videoCount += 1
        except Exception:
            pass
        
get_youtube_titles_from_conversation()

Lets make some graphs(finally)! 🎉 

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

x_axe = list(messages_per_month_dict.keys())
y_axe = list(messages_per_month_dict.values())

x_axe.reverse()

y_axe = [item[0] for item in y_axe]
y_axe.reverse()

title = 'Month graph 🎈 {}'.format(msg_json['title'])

plotly.offline.plot({
    "data": [go.Scatter(x=x_axe, y=y_axe)],
    "layout": go.Layout(title=title)
}, auto_open=True)

Per sender message graph 🎈

In [ ]:
import plotly
import plotly.plotly as py
import plotly.graph_objs as go

import pandas as pd

title = 'Month graph 🎈 {}'.format(msg_json['title'])

x_axe = list(messages_per_month_dict.keys())
x_axe.reverse()

y_sender = list(messages_per_month_dict_A.values())
y_you = list(messages_per_month_dict_B.values())

y_sender.reverse()
y_you.reverse()

layout = go.Layout(
    title=title,
    xaxis=dict(
        title='Dates',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    ),
    yaxis=dict(
        title='Messages',
        titlefont=dict(
            family='Courier New, monospace',
            size=18,
            color='#7f7f7f'
        )
    )
)

sender_mssg = go.Scatter(
                x=x_axe,
                y=y_sender,
                name = msg_json['title'],
                line = dict(color = '#17BECF'),
                opacity = 0.8)

you_mssg = go.Scatter(
                x=x_axe,
                y=y_you,
                name = "You",
                line = dict(color = '#7F7F7F'),
                opacity = 0.8)

data = [sender_mssg,you_mssg]

plotly.offline.plot({
    "data": data,
    "layout": go.Layout(layout),
}, auto_open=True, filename='{}_month.html'.format(msg_json['title']))